In [1]:
import sys, os
from Scripts import train_metric_learning, run_metric_learning_inference, train_gnn
import yaml

/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-cori/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CONFIG = 'pipeline_config.yaml'

# 0. Download Data

In [ ]:
!mkdir datasets
!wget https://portal.nersc.gov/cfs/m3443/dtmurnane/TrackML_Example/trackml_quickstart_dataset.tar.gz -O datasets/trackml_quickstart_dataset.tar.gz
!tar -xvf datasets/trackml_quickstart_dataset.tar.gz -C datasets

# 1. Train Metric Learning

In [22]:
with open(CONFIG, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
print('Common configurations:')
print(yaml.dump(config['common_configs']))
print('Metric learning configurations:')
print(yaml.dump(config['metric_learning_configs']))

Common configurations:
artifact_directory: artifacts
experiment_name: trackml_quickstart_1
gpus: 1
max_epochs: 10

Metric learning configurations:
activation: Tanh
cell_channels: 9
emb_dim: 12
emb_hidden: 1024
factor: 0.5
input_dir: datasets/quickstart_example_1GeV
knn: 100
lr: 0.001
margin: 0.1
nb_layer: 4
output_dir: datasets/quickstart_metric_learning_processed
patience: 30
points_per_batch: 100000
pt_background_cut: 1.0
pt_signal_cut: 1.0
r_test: 0.1
r_train: 0.1
r_val: 0.1
randomisation: 1
regime:
- rp
- hnm
- ci
- norm
spatial_channels: 3
train_split:
- 80
- 10
- 10
true_edges: modulewise_true_edges
warmup: 5
weight: 1



In [24]:
train_metric_learning(CONFIG)

Multiprocessing is handled by SLURM.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name    | Type       | Params
---------------------------------------
0 | network | Sequential | 3.2 M 
---------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.730    Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-cori/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-cori/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:06<00:06,  6.79s/it]

/global/cfs/cdirs/m3443/usr/pmtuan/Tracking-ML-Exa.TrkX/Pipelines/TrackML_Example/LightningModules/Embedding/embedding_base.py:300: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  eff = torch.tensor(cluster_true_positive / cluster_true)
/global/cfs/cdirs/m3443/usr/pmtuan/Tracking-ML-Exa.TrkX/Pipelines/TrackML_Example/LightningModules/Embedding/embedding_base.py:301: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pur = torch.tensor(cluster_true_positive / cluster_positive)
/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-cori/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found 

...


Epoch 0:  99%|█████████▉| 89/90 [00:19<00:00,  4.54it/s, loss=0.01, v_num=2239946]

/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-cori/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 13937. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(



Epoch 1: 100%|██████████| 90/90 [00:19<00:00,  4.51it/s, loss=0.01, v_num=2239946]

/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-cori/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 14665. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 1:  89%|████████▉ | 80/90 [00:37<00:04,  2.15it/s, loss=0.00999, v_num=2239946]
Validation: 0it [00:00, ?it/s]
Epoch 2:  89%|████████▉ | 80/90 [00:57<00:07,  1.39it/s, loss=0.00993, v_num=2239946] 
Validation: 0it [00:00, ?it/s]
Epoch 3:  89%|████████▉ | 80/90 [01:17<00:09,  1.03it/s, loss=0.00999, v_num=2239946] 
Validation: 0it [00:00, ?it/s]
Epoch 4:  89%|████████▉ | 80/90 [01:37<00:12,  1.22s/it, loss=0.00991, v_num=2239946] 
Validation: 0it [00:00, ?it/s]
Epoch 5:  89%|████████▉ | 80/90 [01:57<00:14,  1.47s/it, loss=0.00992, v_num=2239946]  
Validation: 0it [00:00, ?it/s]
Epoch 6:  89%|████████▉ | 80/90 [02:17<00:17,  1.71s/it, loss=0.00928, v_num=2239946]  
Validation: 0it [00:00, ?it/s]
Epoch 7:  89%|████████▉ | 80/90 [02:34<00:19,  1.93s/it, loss=0.00956, v_num=2239946]  
Validation: 0it [00:00, ?it/s]
Epoch 8:  89%|████████▉ | 80/90 [02:46<00:20,  2.09s/it, loss=0.00917, v_num=2239946]  
Validation: 0it [00:00, ?it/s]
Epoch 9:  89%|████████▉ | 80/90 [02:59<00:22,  2.24s/

# 2. Construct graphs from metric learning inference

In [4]:
run_metric_learning_inference(CONFIG)

Training finished, running inference to build graphs...


  0%|          | 0/80 [00:00<?, ?it/s]/global/cfs/cdirs/m3443/usr/pmtuan/Tracking-ML-Exa.TrkX/Pipelines/TrackML_Example/LightningModules/Embedding/embedding_base.py:300: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  eff = torch.tensor(cluster_true_positive / cluster_true)
/global/cfs/cdirs/m3443/usr/pmtuan/Tracking-ML-Exa.TrkX/Pipelines/TrackML_Example/LightningModules/Embedding/embedding_base.py:301: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pur = torch.tensor(cluster_true_positive / cluster_positive)
100%|██████████| 10/10 [00:05<00:00,  1.88it/s]


# 3. Train graph neural networks

In [3]:
train_gnn(CONFIG)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | network | Sequential | 3.2 M 
---------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.730    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-cori/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-cori/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/global/cfs/cdirs/m3443/usr/pmtuan/Tracking-ML-Exa.TrkX/Pipelines/TrackML_Example/LightningModules/Embedding/embedding_base.py:300: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  eff = torch.tensor(cluster_true_positive / cluster_true)
/global/cfs/cdirs/m3443/usr/pmtuan/Tracking-ML-Exa.TrkX/Pipelines/TrackML_Example/LightningModules/Embedding/embedding_base.py:301: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pur = torch.tensor(cluster_true_positive / cluster_positive)


...

Epoch 0:  99%|█████████▉| 89/90 [00:40<00:00,  2.21it/s, loss=0.01, v_num=2239949]

/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-cori/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 13937. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 0: 100%|██████████| 90/90 [00:40<00:00,  2.21it/s, loss=0.01, v_num=2239949]

/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx-cori/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 14665. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 9: 100%|██████████| 90/90 [06:13<00:00,  4.15s/it, loss=0.00849, v_num=2239949]  
